In [15]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from ipywidgets import interact

from ml_collections import ConfigDict
from models.ETD_KT_CM_JAX_Vectorised import *#ETD_KT_CM_JAX_Vectorised, initial_condition, KS_params, KDV_params
from filters import resamplers
from filters.filter import ParticleFilter

In [16]:
signal_params = ConfigDict(KS_params_2)
signal_params.update(E=1,method='ETDRK4', nx = 128)

ensemble_params = ConfigDict(KS_params_2)
ensemble_params.update(method='SS_ETDRK4_SSP33', nx = 128, S=10, P=1)
ensemble_params.update(E=100)
ensemble_params.update(sigma=0.5)

signal_model = ETD_KT_CM_JAX_Vectorised(signal_params)
ensemble_model = ETD_KT_CM_JAX_Vectorised(ensemble_params)

initial_signal = initial_condition(signal_model.x, signal_params.E, signal_params.ic)
initial_ensemble = initial_condition(ensemble_model.x, ensemble_params.E, ensemble_params.ic) 
#initial_ensemble += 0.01 * jax.random.normal(jax.random.PRNGKey(56), initial_ensemble.shape) # adding this is totally "unphysical"

available_resamplers = ", ".join(resamplers.keys())
print(available_resamplers)


multinomial, systematic, no_resampling, default


In [17]:
pf_systematic = ParticleFilter(
    n_particles = ensemble_params.E,
    n_steps = 1,
    n_dim = initial_signal.shape[-1],
    forward_model = ensemble_model,
    signal_model = signal_model,
    sigma = 0.5,
    seed = 11,
    resampling='systematic',
)
# pf_no_resampling = ParticleFilter(
#     n_particles = ensemble_params.E,
#     n_steps = 1,
#     n_dim = initial_signal.shape[-1],
#     forward_model = ensemble_model,
#     signal_model = signal_model,
#     sigma = 0.01,
#     seed = 11,
#     resampling='no_resampling',
# )

In [18]:
final, all = pf_systematic.run(initial_ensemble, initial_signal, 1000) #the final input is scan length? 

In [19]:
print(type(all))
print(len(all))
print(all[0].shape) # particles
print(all[1].shape) # signal
#out shape is (n_total, n_particles, n_dim)

<class 'tuple'>
3
(1000, 100, 128)
(1000, 1, 128)


In [20]:
print(initial_signal.shape)
signal = jnp.concatenate([initial_signal[None,...], all[1]], axis=0)
print(signal.shape)
particles = jnp.concatenate([initial_ensemble[None,...], all[0]], axis=0)
print(particles.shape)

(1, 128)
(1001, 1, 128)
(1001, 100, 128)


In [21]:
def plot(da_step):
    plt.plot(signal_model.x, signal[da_step,0,:], color='red')
    plt.plot(signal_model.x, particles[da_step,:,:].T, color='k', linewidth=0.01)
    plt.legend(['signal', 'particles'])
    plt.show()

interact(plot, da_step=(0, 101))

interactive(children=(IntSlider(value=50, description='da_step', max=101), Output()), _dom_classes=('widget-in…

<function __main__.plot(da_step)>